In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pickle
import json
import os
import re
import sys

from tqdm import tqdm
src_dir = os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)), 'src')
sys.path.append(src_dir)

In [7]:
datasets = {}

suffix = ".jsonl"

dataset_dir = "/Users/cherian/Projects/conformal-safety/data/MedLFQAv2"
for path in os.listdir(dataset_dir):
    dataset_name = path[:-len(suffix)]
    with open(os.path.join(dataset_dir, path), 'r') as fp:
        datasets[dataset_name] = [json.loads(line) for line in fp.readlines()]

In [8]:
datasets.keys()

dict_keys(['medication_qa', 'kqa_golden', 'live_qa', 'healthsearch_qa', 'kqa_silver_wogold'])

In [ ]:
def remove_specific_leading_chars(input_string):
    # Remove leading commas
    input_string = re.sub(r'^,+', '', input_string)
    # Remove numbers followed by a comma
    return re.sub(r'^\d+,+', '', input_string)
    
all_points = len([pt for dataset in datasets.values() for pt in dataset])

for dataset_name, dataset in datasets.items():
    for pt in dataset:
        parsed_prompt = remove_specific_leading_chars(pt['Question'])
        if parsed_prompt != pt["Question"]:
            print(pt["Question"], parsed_prompt)
        pt["Question"] = parsed_prompt.strip()



In [ ]:
removal = {}
for name, dataset in datasets.items():
    redundant_prompts = set()
    indices_to_remove = []
    for i, dat in enumerate(dataset):
        if dat['Question'] in redundant_prompts:
            indices_to_remove.append(i)
        else:
            redundant_prompts.add(dat["Question"])
    removal[name] = indices_to_remove
    

In [ ]:
filtered_datasets = {}

for name, dataset in datasets.items():
    new_dataset = [dat for i, dat in enumerate(dataset) if i not in removal[name]]
    filtered_datasets[name] = new_dataset



In [ ]:
removal = {}
redundant_prompts = {}

for name, dataset in filtered_datasets.items():
    print(name)
    for i, dat in enumerate(dataset):
        if dat["Question"] in redundant_prompts:
            redundant_prompts[dat["Question"]] += 1
        else:
            redundant_prompts[dat["Question"]] = 1


for name, dataset in filtered_datasets.items():
    indices_to_remove = []
    if name == 'kqa_golden' or name == 'live_qa': continue
    for i, dat in enumerate(dataset):
        if redundant_prompts[dat['Question']] > 1:
            indices_to_remove.append(i)
    removal[name] = indices_to_remove

filtered_datasets_2 = {}

for name, dataset in filtered_datasets.items():
    new_dataset = [dat for i, dat in enumerate(dataset) if i not in removal.get(name, [])]
    filtered_datasets_2[name] = new_dataset

In [ ]:
data_path = "/Users/cherian/Projects/conformal-safety/data/MedLFQAv2"

for name, dataset in filtered_datasets_2.items():
    filepath = os.path.join(data_path, f"{name}.jsonl")
    with open(filepath, 'w') as outfile:
        for dat in dataset:
            json.dump(dat, outfile)
            outfile.write('\n')

In [ ]:
data_path = "/Users/cherian/Projects/conformal-safety/data/MedLFQAv2"

datasets = {}

suffix = ".jsonl"

for filename in os.listdir(data_path):
    dataset_name = filename[:-len(suffix)]
    filepath = os.path.join(data_path, filename)
    # print(filepath)
    with open(filepath, 'r') as fp:
        datasets[dataset_name] = [json.loads(line) for line in fp.readlines()]

In [ ]:
prompts = []
for _, dataset in datasets.items():
    prompts.extend([pt['Question'] for pt in dataset])

In [15]:
datasets['medication_qa'][29]

{'Question': 'what is the imprint on metoprolol succ., 50 mg',
 'Free_form_answer': '50 mg tablets: White, round, coated tablets debossed with Andrx logo and “831” on one side and scored on the other side.',
 'Must_have': ['Metoprolol succinate 50 mg tablets are white, round, coated tablets debossed with Andrx logo and “831” on one side and scored on the other side.'],
 'Nice_to_have': ["None as the question specifically asks for the imprint and doesn't require any supplementary information."]}